In [70]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

importlib.reload(utils)
importlib.reload(rt)

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [2]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [3]:
analysis_date = dt.date(2022, 2, 8) ##tuesday, new tables

In [4]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [5]:
# d3 = [273, 372, 221]

In [6]:
pbar = tqdm()

0it [00:00, ?it/s]

In [ ]:
for agency in air_joined.calitp_itp_id.unique():
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        rt_day.set_filter(start_time='15:00', end_time='19:00')
        rt_day.filter_formatted = ', PM Peak, Feb 08 (Tue)' ##TODO fix hardcode
        rt_day.agency_name = (air_joined >> filter(_.calitp_itp_id == agency)).Name.iloc[0]
        m = rt_day.segment_speed_map(how='low_speeds', colorscale=colorscale, size = [1300, 700])
        m.save(f'./speedmaps/{agency}_02_08_pm_peak.html')
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        print(e)

calculating for agency: 380...


/opt/conda/lib/python3.9/site-packages/siuba/sql/utils.py:52: SAWarning: Dialect bigquery:bigquery will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)


gs://calitp-analytics-data/data-analyses/rt_delay/cached_views/
stops_380_2022-02-08


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:36: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


rt failed for agency 380
vehicle positions trip ids not in schedule
calculating for agency: 300...
gs://calitp-analytics-data/data-analyses/rt_delay/cached_views/
stops_300_2022-02-08


...

found_parquet
could not generate delays for trip 604-1
Columns must be same length as key
could not generate delays for trip 603-5
Columns must be same length as key
could not generate delays for trip 603-2
Columns must be same length as key
could not generate delays for trip 604-4
zero-size array to reduction operation maximum which has no identity
could not generate delays for trip 604-3
Columns must be same length as key
could not generate delays for trip 603-3
Columns must be same length as key
could not generate delays for trip 600-5
Columns must be same length as key
could not generate delays for trip 402-1
Columns must be same length as key
could not generate delays for trip 404-2
Columns must be same length as key
view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': None, 'direction_id': None, 'direction': None}


/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:503: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



complete for agency: 75
calculating for agency: 4...
gs://calitp-analytics-data/data-analyses/rt_delay/cached_views/
stops_4_2022-02-08


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:36: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:36: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a 

vehicle positions gdf must not be empty


In [73]:
# rt_day = rt.OperatorDayAnalysis(182, analysis_date, pbar)
rt_day.set_filter(start_time='15:00', end_time='19:00')
rt_day.filter_formatted = ', PM Peak, Feb 08 (Tue)' ##TODO fix hardcode
rt_day.agency_name = (air_joined >> filter(_.calitp_itp_id == 182)).Name.iloc[0]
m = rt_day.segment_speed_map(how='low_speeds', colorscale=colorscale, size = [1300, 700])

view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': None, 'direction_id': None, 'direction': None}
speed above 80 for shape 2580053_DEC21, dropping
speed above 80 for shape 550284_DEC21, dropping
speed above 80 for shape 2050085_DEC21, dropping
negative speed for shape 2050085_DEC21, dropping
speed above 80 for shape 2050087_DEC21, dropping
negative speed for shape 2050087_DEC21, dropping
speed above 80 for shape 300807_DEC21, dropping
speed above 80 for shape 2320087_DEC21, dropping
negative speed for shape 2320087_DEC21, dropping
speed above 80 for shape 1670095_DEC21, dropping
speed above 80 for shape 2440007_DEC21, dropping
negative speed for shape 2370024_DEC21, dropping
negative speed for shape 1620001_DEC21, dropping
speed above 80 for shape 1110380_DEC21, dropping
speed above 80 for shape 100732_DEC21, dropping
speed above 80 for shape 1650198_DEC21, dropping
speed above 80 for shape 1200095_DEC21, dropping
negative speed for shape 1

TopologyException: assigned depths do not match at 135120.06470010104 -418368.46182682936
TopologyException: assigned depths do not match at 135120.06470010104 -418368.46182682936
TopologyException: assigned depths do not match at 135120.06470010104 -418368.46182682936
/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:511: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [74]:
m.save(f'./speedmaps/{182}_02_08_pm_peak.html')

In [ ]:
air_joined['url'] = air_joined.apply(lambda x:
                    f'http://docs.calitp.org/data-analyses/rt_delay/speedmaps/{x.calitp_itp_id}_02_08_pm_peak.html',
                                    axis = 1)

air_joined.to_csv('linked.csv')

In [11]:
air_joined

,Name,calitp_itp_id,Reporting Category,Caltrans District,GTFS Static Status,rt_status,Missing Static,Services Needing Alerts,Services Needing TripUpdates or VehiclePositions,calitp_url_number,min_timestamp,max_timestamp,url
0,Ventura County Transportation Commission,380,Core,07 - Los Angeles,Static OK,RT OK,None,Valley Express Dial-A-Ride,Valley Express Dial-A-Ride,0,2022-02-03 20:07:26,2022-02-10 15:59:49,http://docs.calitp.org/data-analyses/rt_delay/...
1,City of Santa Monica,300,Core,07 - Los Angeles,Static OK,RT OK,None,"Mobility on Demand Everyday, Santa Monica Coll...","Mobility on Demand Everyday, Santa Monica Coll...",0,2022-02-06 15:55:20,2022-02-10 15:59:35,http://docs.calitp.org/data-analyses/rt_delay/...
2,Los Angeles County Metropolitan Transportation...,182,Core,07 - Los Angeles,Static OK,RT OK,None,LA Metro Micro,LA Metro Micro,1,2022-02-06 15:58:17,2022-02-10 15:59:45,http://docs.calitp.org/data-analyses/rt_delay/...
3,City of Pasadena,243,Core,07 - Los Angeles,Static OK,RT OK,None,Pasadena Dial-A-Ride,Pasadena Dial-A-Ride,0,2022-02-06 15:57:19,2022-02-10 15:59:25,http://docs.calitp.org/data-analyses/rt_delay/...
4,City of Santa Clarita,295,Core,07 - Los Angeles,Static OK,RT OK,None,"Go! Santa Clarita, Santa Clarita Summer Trolly...","Go! Santa Clarita, Santa Clarita Summer Trolly...",0,2022-02-06 15:59:20,2022-02-09 17:34:57,http://docs.calitp.org/data-analyses/rt_delay/...
5,Long Beach Transit,170,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,"AquaLink, UCLA/Westwood Commuter Express","Galaxy Express, AquaLink, UCLA/Westwood Commut...","Galaxy Express, AquaLink, UCLA/Westwood Commut...",0,2022-02-06 15:54:38,2022-02-10 15:59:16,http://docs.calitp.org/data-analyses/rt_delay/...
6,City of Burbank,45,Core,07 - Los Angeles,Static OK,RT OK,None,Burbank Bus Senior and Disabled,Burbank Bus Senior and Disabled,0,2015-01-04 23:08:07,2022-02-10 15:59:52,http://docs.calitp.org/data-analyses/rt_delay/...
7,City of Commerce,75,Core,07 - Los Angeles,Static OK,RT OK,None,Commerce Dial-A-Ride,Commerce Dial-A-Ride,0,2022-02-06 14:46:27,2022-02-10 15:59:49,http://docs.calitp.org/data-analyses/rt_delay/...
8,Alameda-Contra Costa Transit District,4,Core,04 - Oakland,Static OK,RT OK,None,East Bay Paratransit,East Bay Paratransit,1,2022-02-06 15:59:47,2022-02-10 15:59:34,http://docs.calitp.org/data-analyses/rt_delay/...
9,Solano County Transit,310,Core,04 - Oakland,Static OK,RT OK,None,SolTrans ADA Paratransit,SolTrans ADA Paratransit,1,2022-02-06 16:00:00,2022-02-10 15:59:30,http://docs.calitp.org/data-analyses/rt_delay/...
